In [31]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, KFold
from surprise import accuracy
from surprise import KNNBasic
from collections import defaultdict

In [2]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/marrechea/.surprise_data/ml-100k


In [4]:
svd = SVD()

In [10]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9341  0.9393  0.9333  0.9356  0.9338  0.9352  0.0022  
MAE (testset)     0.7382  0.7426  0.7346  0.7341  0.7372  0.7373  0.0030  
Fit time          3.36    3.33    3.33    3.32    3.33    3.34    0.01    
Test time         0.12    0.12    0.12    0.12    0.13    0.12    0.00    


{'test_rmse': array([0.93408422, 0.9392849 , 0.93330771, 0.93561144, 0.93382852]),
 'test_mae': array([0.73815788, 0.74257977, 0.73464552, 0.73405795, 0.73721797]),
 'fit_time': (3.357534885406494,
  3.3349175453186035,
  3.330157995223999,
  3.324648141860962,
  3.332836866378784),
 'test_time': (0.1229705810546875,
  0.12109088897705078,
  0.12104153633117676,
  0.12071967124938965,
  0.1256422996520996)}

In [26]:
train = data.build_full_trainset()
svd.fit(trainset)

In [28]:
test = trainset.build_testset()
predicts = svd.test(test)
accuracy.rmse(predicts, verbose=True)

RMSE: 0.6757


0.6756587931722513

In [18]:
kf = KFold(n_splits=10)

In [17]:
for trainset, testset in kf.split(data):
    svd.fit(trainset)
    predictions = svd.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.9292
RMSE: 0.9256
RMSE: 0.9232
RMSE: 0.9203
RMSE: 0.9383
RMSE: 0.9405
RMSE: 0.9356
RMSE: 0.9211
RMSE: 0.9291
RMSE: 0.9297


In [21]:
knn = KNNBasic()
trainset = data.build_full_trainset()
knn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [24]:
pred = knn.predict(196, 303, r_ui =4, verbose=True)

user: 196        item: 303        r_ui = 4.00   est = 3.53   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


In [29]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [32]:
top_n = get_top_n(predicts)
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

196 ['286', '285', '8', '663', '655', '428', '1007', '306', '692', '251']
186 ['98', '79', '71', '356', '300', '566', '226', '385', '306', '117']
22 ['50', '174', '186', '173', '153', '144', '181', '187', '172', '127']
244 ['9', '56', '64', '318', '276', '168', '151', '357', '89', '154']
166 ['313', '258', '300', '328', '315', '322', '347', '751', '984', '343']
298 ['318', '174', '427', '496', '651', '50', '357', '194', '603', '498']
115 ['654', '23', '357', '100', '475', '137', '192', '56', '127', '302']
253 ['318', '64', '496', '22', '427', '12', '98', '79', '190', '483']
305 ['408', '169', '923', '709', '357', '483', '480', '207', '199', '251']
6 ['474', '427', '519', '484', '511', '302', '100', '127', '134', '185']
62 ['50', '56', '172', '238', '173', '98', '318', '100', '89', '127']
286 ['289', '57', '465', '272', '483', '70', '251', '14', '315', '301']
200 ['174', '50', '172', '318', '173', '313', '496', '98', '169', '191']
210 ['127', '174', '56', '50', '483', '357', '187', '98'